# Computing causal Shapley values

Here we show how to create a Causal Shapley masker to retrieve causal Shapley values using SHAP.

In [1]:
import xgboost

import shap
from shap.maskers._tabular import Causal as CausalMasker
from shap.maskers._tabular import Independent as IndependentMasker

# get a dataset on income prediction
# X, y = shap.datasets.adult()
X, y = shap.datasets.iris()
# X, y = shap.datasets.diabetes()

# train an XGBoost model (but any other model type would also work)
model = xgboost.XGBClassifier()
model.fit(X, y)

# Define masker
marginal_masker = IndependentMasker(X)
causal_masker = CausalMasker(X)

No causal ordering provided. Consider using the Independent Masker if your goal is to compute marginal Shapley values.
No confounding provided. Assuming that all causal groups contain have confounders present.


In [2]:
import xgboost

import shap

# get a dataset on income prediction
X, y = shap.datasets.adult()
# X, y = shap.datasets.iris()
# X, y = shap.datasets.diabetes()

# train an XGBoost model (but any other model type would also work)
model = xgboost.XGBClassifier()
model.fit(X, y);

In [3]:
from shap.maskers._tabular import Causal as CausalMasker
from shap.maskers._tabular import Independent as IndependentMasker

marginal_masker = IndependentMasker(X)
causal_masker = CausalMasker(X)

No causal ordering provided. Consider using the Independent Masker if your goal is to compute marginal Shapley values.
No confounding provided. Assuming that all causal groups contain have confounders present.


In [4]:
# build an Exact explainer, although we can use any explainer that takes in a masker
# explainer = shap.explainers.Exact(model.predict_proba, marginal_masker)
explainer = shap.explainers.Exact(model.predict_proba, causal_masker)

# explain the model predictions on the given dataset
shap_values = explainer(X[:100])

# get just the explanations for the positive class
shap_values = shap_values[..., 1]
shap.plots.bar(shap_values)

[3.900e+01 7.000e+00 1.300e+01 4.000e+00 1.000e+00 0.000e+00 4.000e+00
 1.000e+00 2.174e+03 0.000e+00 4.000e+01 3.900e+01]
[2147483647         11         10 ...         11         10         11]


AssertionError: mask must have the same shape as features. expected 32561, received (4096,).